# Interaction Tracking Analysis

Code to analyze the output from `interactionTracker.js`: process it to determine the likely interactions and reduce the amount of data, then produce some statistics.

In [1]:
import os
import glob

def get_newest_file(directory, startsWith = "interactions_"):
    # Get list of all files in the directory
    files = glob.glob(os.path.join(directory, '*'))
    files = [file.replace("\\", "/") for file in files]
    
    # Check if the directory is empty
    if not files:
        return None

    # Filter for interaction data.
    if startsWith:
        files = [file for file in files if file.split("/")[-1].startswith(startsWith)]
    
    # Get the newest file based on modification time
    newest_file = max(files, key=os.path.getmtime)
    return newest_file

In [2]:
import os
import json

userprofile = os.path.expanduser("~").replace("\\", "/")
fileDir = userprofile + '/AppData/Roaming/Code/User/workspaceStorage/b03f3379a1447d3620f3d4a069f1d983/grandFileNavigator.grandfilenavigator'
filePath = get_newest_file(fileDir)

windowData = []
with open(filePath) as windowDataFile:
    for windowSwitch in windowDataFile.readlines():
        windowData.append(json.loads(windowSwitch))

print(windowData)

[{'timeStamp': 1746871877456, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '12-46'}, {'timeStamp': 1746871877460, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '12-46', 'targetRange': '10-44'}, {'timeStamp': 1746871880452, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '10-44', 'targetRange': '9-43'}, {'timeStamp': 1746871880454, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '9-43', 'targetRange': '8-43'}, {'timeStamp': 1746871880460, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '8-43', 'targetRange': '7-42'}, {'timeStamp': 1746871880462, 'i

In [3]:
def remove_erroneous(interactionData):
    fixedInteractionData = []

    for interaction in interactionData:
        if interaction["interactionType"] == "ChangeFile":
            if not interaction["targetFilePath"] or interaction["sourceFilePath"] == interaction["targetFilePath"]:
                continue
        fixedInteractionData.append(interaction)
    
    return fixedInteractionData


def remove_duplicates(interactionData):
    cleanedInteractionData = []

    i = 0
    while i < len(interactionData):
        if i == len(interactionData) - 1 or interactionData[i] != interactionData[i+1]:
            cleanedInteractionData.append(interactionData[i])
        i += 1
    
    return cleanedInteractionData

In [4]:
import copy

# Merge scrolls together.
# A scroll is all changes of the visible ranges with less than 250 between them (i.e., covers short interruptions).


def detect_scrolling(interactionData):
    # max time in ms between ChangeVisibleRanges entries to still be considered part of one scroll
    maxTimeBetweenChanges = 250
    scrollingInteractionData = []

    changeRangesInteraction = None
    for interaction in interactionData:
        if interaction["interactionType"] != "ChangeVisibleRanges":
            if changeRangesInteraction is not None:
                scrollingInteractionData.append(changeRangesInteraction)
                changeRangesInteraction = None
            scrollingInteractionData.append(interaction)
            continue
        
        if changeRangesInteraction is None:
            changeRangesInteraction = copy.deepcopy(interaction)
            continue

        isScrollInteraction = changeRangesInteraction["interactionType"] == "Scroll"
        lastInteractionEndTime = changeRangesInteraction["endTime"] if isScrollInteraction else changeRangesInteraction["timeStamp"]
        if interaction["timeStamp"] - lastInteractionEndTime > maxTimeBetweenChanges:
            scrollingInteractionData.append(changeRangesInteraction)
            changeRangesInteraction = None
        else:
            if changeRangesInteraction["interactionType"] == "ChangeVisibleRanges": # wasn't treated as scroll yet
                changeRangesInteraction["interactionType"] = "Scroll"
                changeRangesInteraction["startTime"] = changeRangesInteraction["timeStamp"]
            changeRangesInteraction["endTime"] = interaction["timeStamp"]
            changeRangesInteraction["targetRange"] = interaction["targetRange"]
    
    if changeRangesInteraction is not None:
        scrollingInteractionData.append(changeRangesInteraction)

    return scrollingInteractionData

In [5]:
def combine_edits(interactionData):
    # max time in ms between ChangeVisibleRanges entries to still be considered part of one edit session
    maxTimeBetweenChanges = 2000
    editingInteractionData = []

    editInteraction = None
    for interaction in interactionData:
        if interaction["interactionType"] != "EditFile":
            if editInteraction is not None:
                editingInteractionData.append(editInteraction)
                editInteraction = None
            editingInteractionData.append(interaction)
            continue
        
        if editInteraction is None:
            editInteraction = copy.deepcopy(interaction)
            continue

        isEditingSession = editInteraction["interactionType"] == "EditingSession"
        lastInteractionEndTime = editInteraction["endTime"] if isEditingSession else editInteraction["timeStamp"]
        if interaction["timeStamp"] - lastInteractionEndTime > maxTimeBetweenChanges:
            editingInteractionData.append(editInteraction)
            editInteraction = None
        else:
            if editInteraction["interactionType"] == "EditFile": # wasn't treated as session yet
                editInteraction["interactionType"] = "EditingSession"
                editInteraction["startTime"] = editInteraction["timeStamp"]
            editInteraction["endTime"] = interaction["timeStamp"]

    if editInteraction is not None:
        editingInteractionData.append(editInteraction)

    return editingInteractionData

In [6]:
def process_jumps(interactionData):
    navigationInteractions = ["ClickJumpButton", "ClickStatusBar", "NavigationJump"]
    jumpInteractionData = []

    i = 0
    while i < len(interactionData):
        if interactionData[i]["interactionType"] in navigationInteractions:
            if (i == 121):
                print("i == 121")
            
            triggeredByUiInteraction = interactionData[i]["interactionType"] != "NavigationJump"
            jumpEntryIndex = i+2 if triggeredByUiInteraction else i+1

            combinedEntry = interactionData[jumpEntryIndex]
            combinedEntry["interactionType"] = "NavigationJump"
            combinedEntry["backwards"] = interactionData[i]["backwards"]

            if interactionData[i]["interactionType"] == "ClickJumpButton":
                combinedEntry["origin"] = "SidebarButton"
            elif interactionData[i]["interactionType"] == "ClickStatusBar":
                combinedEntry["origin"] = "StatusBar"
            else:
                combinedEntry["origin"] = "KeyboardShortcut"

            jumpInteractionData.append(combinedEntry)
            i += 3 if triggeredByUiInteraction else 2
        else:
            jumpInteractionData.append(interactionData[i])
            i += 1
        
    return jumpInteractionData

In [7]:
for windowSwitch in windowData:
    print(windowSwitch)

refinedInteractionData = detect_scrolling(windowData)
refinedInteractionData = combine_edits(refinedInteractionData)
refinedInteractionData = remove_erroneous(refinedInteractionData)
refinedInteractionData = remove_duplicates(refinedInteractionData)
refinedInteractionData = process_jumps(refinedInteractionData)

{'timeStamp': 1746871877456, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '12-46'}
{'timeStamp': 1746871877460, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '12-46', 'targetRange': '10-44'}
{'timeStamp': 1746871880452, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '10-44', 'targetRange': '9-43'}
{'timeStamp': 1746871880454, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '9-43', 'targetRange': '8-43'}
{'timeStamp': 1746871880460, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '8-43', 'targetRange': '7-42'}
{'timeStamp': 1746871880462, 'interac

In [8]:
for windowSwitch in refinedInteractionData:
    print(windowSwitch)

{'timeStamp': 1746871877456, 'interactionType': 'Scroll', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '10-44', 'startTime': 1746871877456, 'endTime': 1746871877460}
{'timeStamp': 1746871880454, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '9-43', 'targetRange': '0-35', 'startTime': 1746871880454, 'endTime': 1746871880467}
{'timeStamp': 1746871880944, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '1-35', 'targetRange': '21-55', 'startTime': 1746871880944, 'endTime': 1746871880989}
{'timeStamp': 1746871881339, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\graph_handler.py', 'sourceRange': '22-56', 'targetRange': '36-70', 'startTime': 1746871881339, 'endTime': 1746871881533}
{'timeStamp': 1746871911752, 'interactionType': 'Scroll', 'sourceFilePath': 'c:

## Ratio of VS Code Usage

To detect when VS Code is not open and not take these sections into account when performing the analysis, the open windows are logged using `windowTracker.ps1`

In [14]:
import os

userprofile = os.path.expanduser("~").replace("\\", "/")
filePath = get_newest_file(userprofile, "focused_window_log")

windowData = []
with open(filePath, encoding='utf-16') as windowDataFile:
    for windowSwitch in windowDataFile.readlines():
        windowData.append(windowSwitch)

print(windowData)

['1746873343.58124 - Windows PowerShell\n', '1746873345.59852 - ? InteractionTrackingAnalysis.ipynb - GrandFileNavigator - Visual Studio Code\n', '1746873348.62245 - Windows PowerShell\n']
